In [21]:

import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests
import statsmodels.api as sm
import numpy as np
from sklearn.decomposition import PCA

#Use beautifulSoup to get the S&P 500 tickers and store it in pickle
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

def get_data_from_yahoo(reload_sp500=False):
    #Open pickle
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    #Read stock data from yahoo
    start = dt.datetime(2012, 1, 1)
    end = dt.datetime(2020,1, 1)
    for ticker in tickers: 
        if '.' in ticker:
            array=ticker.split(".")
            ticker= array[0]
        ticker = ticker.rstrip("\n")
        if ticker == 'CARR' or ticker == 'KR'or ticker == 'LHX'or ticker == 'LKQ' or ticker == 'NOC' or ticker == 'OTIS'or ticker == 'UDR' or ticker == 'UNH'or ticker == 'VZ'or ticker == 'VIAC' or ticker == 'BF':
            continue
        if not os.path.exists('stock_dfs\{}.csv'.format(ticker.rstrip("\n"))):
            df = web.DataReader(ticker.rstrip("\n"), start=start, end=end, data_source='yahoo')
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df.to_csv('stock_dfs\{}.csv'.format(ticker.rstrip("\n")))
        else:
            print('Already have {}'.format(ticker))

#Get Adj Close data for each stock and merge the data to one dataframe
def data_Cleaning():
    dataAll = []
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
    for ticker in tickers:
        if '.' in ticker:
            array=ticker.split(".")            
            ticker= array[0]
            print(ticker)
        ticker= ticker.rstrip("\n")  
        if ticker == 'CARR' or ticker == 'KR'or ticker == 'LHX'or ticker == 'LKQ' or ticker == 'NOC' or ticker == 'OTIS'or ticker == 'UDR' or ticker == 'UNH'or ticker == 'VZ'or ticker == 'VIAC'or ticker == 'BF':
            continue
        data = pd.read_csv('stock_dfs\{}.csv'.format(ticker))
        if (ticker=='B'):
            print(data.head())
        data = data.set_index('Date')
        data = data[['Adj Close']].dropna()
        data = data.rename({"Adj Close": ticker},axis='columns')
        dataAll.append(data)
    dataAll=pd.concat(dataAll,axis = 1)
    return dataAll

#Get stock returns from given stock prices
def getReturn(price):
    ret = (price - price.shift(1))/price
    ret = ret.drop(ret.index[0])
    ret = ret.fillna(value = 0)
    return ret

#Calculate Zscores givieng a range of returns
def getZscores(return_in_range, num_factor):
    # Sample data for PCA (smooth it using np.log function)
    sample = return_in_range.replace([np.inf, -np.inf], np.nan)
    sample = sample.dropna(axis = 1,thresh = len(sample)-30)
    mean = sample.mean() 
    #std = sample.std()
    #print('std', std)
    sample = (sample - mean)/sample.std()# Center it column-wise

    # Fit the PCA model for sample data
    sample = pd.DataFrame(sample).fillna(0)
    model = PCA().fit(sample)
    weights = pd.DataFrame(model.components_)
    print('weights',weights)
    #weights_std = weights/std
    #print('weights_std',weights_std)
    # Get the first n_components factors
    factors_ori = np.dot(sample, weights.T)[:,:(num_factor-1)]
    # Add 1's to fit the linear regression (intercept)
    factors = sm.add_constant(factors_ori)
    # Train Ordinary Least Squares linear model for each stock
    OLSmodels = {ticker: sm.OLS(sample[ticker], factors).fit() for ticker in sample.columns}
    # Get the residuals from the linear regression after PCA for each stock
    resids = pd.DataFrame({ticker: model.resid for ticker, model in OLSmodels.items()})
    coefs = pd.DataFrame({ticker: model.params for ticker, model in OLSmodels.items()})
    # Get the Z scores by standarize
    zscores = ((resids - resids.mean()) / resids.std()).iloc[-1] # residuals of the most recent day
    return zscores, coefs,weights,factors

#Using slicing windown to get Zscores Array
def zscoresArr(stopTime, dataAll,num_factor):
    zscoresArr = []
    for t in range(stopTime, len(dataAll)-stopTime-1):
        price_in_range = dataAll[t-1:t+stopTime]
        return_in_range = getReturn(price_in_range)
        zscores = getZscores(return_in_range, num_factor)
        zscoresArr.append(zscores.to_frame())
    zscoresRes = pd.concat(zscoresArr,axis=1)     
    return zscoresRes

def backtesting(zscores, price, signals):
    [ss, sb, cs, cb]=signals
    #initialize original stock position
    position = pd.Series(0, index = price.columns) 
    #initialize money, stockValue and total Value (which is the sum of money and stockValue)
    money = pd.Series(0, index = price.index)
    stockValue = pd.Series(0, index = price.index)
    totalValue = pd.Series(0, index = price.index)
    for i in zscores:
        print(i)
        #get the sell positions, if the zscores are higher than start sell signal
        sell = zscores[i].where(zscores[i] >ss, 0)
        # scaled to make sure that the weights add up to -100%
        sell = sell/sell.sum() 
        # selling stock would increase the money 
        money[i] += sum(sell * price.loc[i])
        # update positions
        position = position - sell
        
        #get the sell positions, if the zscores are higher than start sell signal
        buy = zscores[i].where(zscores[i] <sb, 0)
        # scaled to make sure that the weights add up to 100%
        buy = buy/buy.sum() 
        # buying stock would decrease the money
        money[i] -= sum(buy * price.loc[i]) 
        # update positions
        position = position + buy
        
        #get the clear positions, if the zscores are in between the close buy and close sell signals
        clear = zscores[i].where(zscores[i].between(cb,cs), 0)
        clear = clear.where(clear!=0, 1) #build a boolen mask for positions to clear
        positionToClear = position * clear
        #clear the positions, money value will change according to the positions
        money[i] += sum(positionToClear * price.loc[i])
        #update positions
        position = zscores[i].where(zscores[i].between(cb,cs), 0)
        
        #update stockValue and totalValue
        stockValue[i] = sum(position * price.loc[i])
        totalValue[i] = money[i] + stockValue[i] 
    return money, stockValue, totalValue

def find_Sharpe_Ratio(pnl,r):
    ret = getReturn(pnl)
    ret = find_Return(pnl)
    std = ret.std() * math.sqrt(252)
    return (mean - r)/std

def find_Maximum_Drawdown(pnl):
    ret = find_Return(pnl)
    r = ret.add(1).cumprod()
    dd = r.div(r.cummax()).sub(1)
    mdd = dd.min()
    end = dd.argmin()
    start = r.argmax()
    return mdd

def find_Cumulative_Return(pnl):
    return (pnl.iloc[len(pnl.index)-1] - pnl.iloc[0]) / pnl.iloc[0]

def erase_Nan(price,nan_num):
    price_nan = price.dropna(axis = 1,thresh = len(price)-nan_num)
    return price_nan

    
#Reference:
#Reading the yahoo data https://pythonprogramming.net/sp500-company-price-data-python-programming-for-finance/
#PCA strategy https://www.quantconnect.com/tutorials/strategy-library/mean-reversion-statistical-arbitrage-strategy-in-stocks
#Calculation results https://github.com/YuxiLiuAsana/Statistical-Arbitrage-Avellaneda-/blob/master/main.py


In [ ]:
    
def main():
    stopTime = 252 #for slicing window
    num_factor = 10
    ss = 1.25 #start to sell signal
    sb = -1.25 #start to buy signal
    cs = 0.25 #close to sell signal
    cb = -0.25 #close to buy signal
    r = 0.01 #market rate
    signals = [ss, sb, cs, cb]
    
    get_data_from_yahoo()
    dataAll= data_Cleaning()
    zscores = zscoresArr(stopTime, dataAll.fillna(0),num_factor) 
    money, stockValue, totalValue = backtesting(zscores, dataAll.fillna(0), signals)
    #Calculation resultsd
    sharpe_ratio = find_Sharpe_Ratio(totalValue,r)
    maximum_drawdowns = find_Maximum_Drawdown(totalValue)
    cumulative_return = find_Cumulative_Return(totalValue)
    print('Cumulative return:', cumulative_return, 'Sharpe_Ratio:', sharpe_Ratio, 'Maximum_drawdowns: ',maximum_drawdowns)
if __name__ == "__main__":
    main()    
    

In [23]:
    start_val=100
    delay = 252
    leverage = 1
    window = 60
    num_factor = 15
    sbo = 2
    sso = 2
    sbc = 0.75
    ssc = 0.5
    price_original = pd.read_pickle('pickle/price_original.pkl')
    pnl = pd.Series(start_val, index = price_original.index[delay-1:])
    position_stock = pd.DataFrame(0,columns = price_original.columns, index = ['stock']+[range(num_factor)])
    position_stock_before = pd.Series(0, index = price_original.columns)

In [24]:

        for t in range(delay-1,len(price_original.index)-1):
            #find the window price of all stocks
            price_t = price_original[(t-window):(t+1)]
            print(price_t)
            # eliminate the nan
            price_t = erase_Nan(price_t,0)
            #find the return of this period
            ret_t = getReturn(price_t)
            # find the residual of this period
            #ret_factorret_t = ret_factorret[(t-window+1):(t+1)]
            #res_t, coef_t = al.find_Residue(ret_t,ret_factorret_t)
            # find the s-score of the target stocks of this period
            #target = al.find_Target_sscore(res_t, k)
            target, coef_t,weight,factors=getZscores(ret_t, num_factor)
            # find the strategy for this time period:
            for i in position_stock.columns:
                if not i in target.index :
                    if position_stock[i]['stock'] != 0:
                        position_stock[i] = 0
                else:
                    if position_stock[i]['stock'] == 0:
                        if target[i] < -sbo:
                            position_stock[i]['stock'] = leverage
                            position_stock[i][1:] = -leverage * coef_t[i]
                        elif target[i] > sso:
                            position_stock[i]['stock'] = - leverage
                            position_stock[i][1:] = leverage * coef_t[i]
                    elif position_stock[i][0] >0 and target[i] > -ssc:
                        position_stock[i] = 0
                    elif position_stock[i][0] <0 and target[i] < sbc:
                        position_stock[i] = 0
            # calculate the pnl for the next period
            #dps_t = dps_original.iloc[t+1]
            pri_t = price_original.iloc[t+1]
            #temp = (dps_t/pri_t).fillna(0)
            position_stock_temp = pd.Series(0,index = price_original.columns)
            fac_sum = position_stock.sum(axis = 1)[1:]
            for i in weight.columns:
                position_stock_temp = sum(weight[i] * fac_sum)
            position_stock_temp = position_stock_temp + position_stock.iloc[0]
            change = sum(abs(position_stock_temp - position_stock_before))
            position_stock_before = position_stock_temp
            pnl.iloc[t-delay + 2] = pnl.iloc[t-delay + 1] * ( 1 + r /252.0) + np.dot(position_stock.loc['stock'], ret_original.iloc[t]) + np.dot(position_stock.sum(axis = 1)[1:], ret_factorret.iloc[t]) - position_stock.sum().sum() * r /252.0 - change * tran_cost
            print (pnl.iloc[t-delay + 2])
        pnl.to_pickle('pickle/pnl.pkl')
        print ('pnl.pkl created')


                  MMM        ABT       ABBV       ABMD        ACN       ATVI  \
2012-10-04  28.342432  26.014153  26.014153  26.014153  26.014153  26.014153   
2012-10-05  28.299570  25.788523  25.788523  25.788523  25.788523  25.788523   
2012-10-08  28.085230  25.655792  25.655792  25.655792  25.655792  25.655792   
2012-10-09  28.313858  25.317347  25.317347  25.317347  25.317347  25.317347   
2012-10-10  27.863758  24.826263  24.826263  24.826263  24.826263  24.826263   
...               ...        ...        ...        ...        ...        ...   
2012-12-27  26.629057  27.035898  27.035898  27.035898  27.035898  27.035898   
2012-12-28  26.830133  26.497042  26.497042  26.497042  26.497042  26.497042   
2012-12-31  28.022266  27.235477  27.235477  27.235477  27.235477  27.235477   
2013-01-02  27.512375  27.860807  27.860807  27.860807  27.860807  27.860807   
2013-01-03  26.909130  27.960592  27.960592  27.960592  27.960592  27.960592   

                 ADBE        AMD       

ValueError: cannot set using a slice indexer with a different length than the value